In [4]:
# Create the directory if it doesn't exist
mkdir -p ./end2end_stealing/pretrained_weights

# Download the pretrained weights
wget https://dl.fbaipublicfiles.com/simsiam/models/100ep/pretrain/checkpoint_0099.pth.tar -O ./end2end_stealing/pretrained_weights/checkpoint_0099.pth.tar



SyntaxError: invalid decimal literal (697284317.py, line 5)

In [ ]:
!unzip zipped_outputs.zip -d images/

Archive:  zipped_outputs.zip
 extracting: images/output/100527.png  
 extracting: images/output/100683.png  
 extracting: images/output/101847.png  
 extracting: images/output/102223.png  
 extracting: images/output/102503.png  
 extracting: images/output/102519.png  
 extracting: images/output/102591.png  
 extracting: images/output/102836.png  
 extracting: images/output/10296.png  
 extracting: images/output/10303.png  
 extracting: images/output/103218.png  
 extracting: images/output/103477.png  
 extracting: images/output/103601.png  
 extracting: images/output/103683.png  
 extracting: images/output/103790.png  
 extracting: images/output/103966.png  
 extracting: images/output/105409.png  
 extracting: images/output/105941.png  
 extracting: images/output/105997.png  
 extracting: images/output/106027.png  
 extracting: images/output/106502.png  
 extracting: images/output/106574.png  
 extracting: images/output/106691.png  
 extracting: images/output/10713.png  
 extracting: i

In [ ]:
import torch
from torchvision.models import resnet50
import torchvision.models as models
import torch.nn as nn
import torchvision.datasets as datasets
from typing import Tuple
import torchvision.transforms as transforms

In [ ]:
unzip zipped_outputs.zip -d images/

In [ ]:
def create_model(device, last_layer=nn.Linear(2048, 512)):
  model = models.__dict__["resnet50"]().to(device)
  checkpoint = torch.load("end2end_stealing/pretrained_weights/checkpoint_0099.pth.tar", map_location=device)
  state_dict = checkpoint["state_dict"]
  for k in list(state_dict.keys()):
      # retain only encoder up to before the embedding layer
      if k.startswith("module.encoder") and not k.startswith("module.encoder.fc"):
          # remove prefix
          state_dict[k[len("module.encoder.") :]] = state_dict[k]
      # delete renamed or unused k
      del state_dict[k]
  # print("state dict", state_dict.keys())
  model.load_state_dict(state_dict, strict=False)
  # model.fc = torch.nn.Identity()
  for p in model.parameters():
    p.requires_grad = False
  model.fc = last_layer
  model = model.to(device)
  model.fc.requires_grad = True
  return model

In [ ]:
from torch.utils.data import Dataset
import pandas as pd
from PIL import Image
import torch
import os
import ast
class MyDataset(Dataset):
    def __init__(self, df, transform=None):
        self.data_frame = df
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Construct the full path to the file
        img_name = self.data_frame.iloc[idx, 0]
        image = Image.open(f'images/output/{img_name}').convert("RGB")  # Load the image
        weight = ast.literal_eval(self.data_frame.iloc[idx, 1])
        weight = torch.tensor(weight)

        if self.transform:
            image = self.transform(image)
        return image, weight

In [ ]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("images/output/embeddings.csv")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


dataset = MyDataset(train_df)
dataset.data_frame = train_df
dataset.transform = transforms.Compose(
                    [
                        transforms.RandomResizedCrop(224),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),
                    ]
                )
train_loader = torch.utils.data.DataLoader(
            dataset,
            drop_last=False,
            shuffle=True,
        )
_ = next(iter(train_loader))

test_dataset = MyDataset(test_df)
test_dataset.data_frame = test_df
test_dataset.transform = transforms.Compose(
                    [
                        transforms.RandomResizedCrop(224),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),
                    ]
                )
test_loader = torch.utils.data.DataLoader(
            test_dataset,
            drop_last=False,
            shuffle=True,
        )
_ = next(iter(test_loader))

In [ ]:
def test_loop(model, loader, device, loss_fn):
  model.eval()
  with torch.no_grad():
    losses = []
    for img, target in loader:
          img = img.to(device)
          target = target.to(device)
          output = model(img)
          if output.dim() == 1:
              output = output.unsqueeze(0)
          if target.dim() == 1:
              target = target.unsqueeze(0)
          be_similar = torch.ones((output.size(0),), dtype=torch.float, device=device)
          loss = loss_fn(output, target, be_similar)
          losses.append(loss.item())
    print("Val", np.mean(losses))

In [ ]:
import numpy as np
#make experiment
many_layer = nn.Sequential(
      nn.Linear(2048, 1024),
      nn.ReLU(),
      nn.Linear(1024, 512)
  )
drop_layer = nn.Sequential(
    nn.Dropout(),
    nn.Linear(2048, 512)
)
device = torch.device("cuda")
model = create_model(device, drop_layer)
cosine_loss = nn.CosineEmbeddingLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)
model.eval()
for ep in range(100):
    losses = []
    mse_losses = []
    for img, target in train_loader:
        img = img.to(device)
        target = target.to(device)
        output = model(img)
        if output.dim() == 1:
            output = output.unsqueeze(0)
        if target.dim() == 1:
            target = target.unsqueeze(0)
        be_similar = torch.ones((output.size(0),), dtype=torch.float, device=device)
        loss = cosine_loss(output, target, be_similar)
        losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    print("Train: ", np.mean(losses))
    test_loop(model, test_loader, device, cosine_loss)

losses

Train:  0.2504248446971178
Val 0.22213400691747665
Train:  0.19913313448429107
Val 0.2112161123752594
Train:  0.18454779893159867
Val 0.19834309041500092
Train:  0.17578127577900887
Val 0.19374141186475755
Train:  0.16710432760417462
Val 0.196565226316452
Train:  0.16157939970493318
Val 0.1916026622056961
Train:  0.15623399533331395
Val 0.19042638421058655
Train:  0.15714104048907757
Val 0.18244217544794084
Train:  0.15053618617355824
Val 0.18655271977186202
Train:  0.14926242716610433
Val 0.18462553054094313
Train:  0.14877284295856952
Val 0.1829777130484581
Train:  0.1464722765982151
Val 0.18458991587162019
Train:  0.1435161118209362
Val 0.18578025609254836
Train:  0.1431485204398632
Val 0.18669009059667588
Train:  0.1404859684407711
Val 0.18738575667142868
Train:  0.13910841025412082
Val 0.1773661470413208
Train:  0.1376983004063368
Val 0.17955531805753708
Train:  0.1337747570872307
Val 0.18193773567676544
Train:  0.13620767064392567
Val 0.18195482432842255
Train:  0.133165689706802

In [ ]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 75.7 MB/s eta 0:00:00


In [ ]:
torch.onnx.export(
    model,
    torch.randn(1, 3, 32, 32).cuda(),
    "second_model.onnx",
    export_params=True,
    input_names=["x"],
)

In [ ]:
import requests
def model_stealing_submission():
    SERVER_URL = "http://34.71.138.79:9090"
    ENDPOINT = "/modelstealing/submit"
    URL = SERVER_URL + ENDPOINT

    TEAM_TOKEN = "DdE56eC1EvRwWcuy"

    with open('second_model.onnx', "rb") as onnx_file:
        response = requests.post(
            URL, files={"file": onnx_file}, headers={"token": TEAM_TOKEN}
        )

        if response.status_code == 200:
            return response.content["score"]
        else:
            raise Exception(f"Request failed. Status code: {response.status_code}, content: {response.content}")

In [ ]:
model_stealing_submission()

TypeError: byte indices must be integers or slices, not str

# Create the directory if it doesn't exist
!mkdir -p ./end2end_stealing/pretrained_weights

# Download the pretrained weights
!wget https://dl.fbaipublicfiles.com/simsiam/models/100ep/pretrain/checkpoint_0099.pth.tar -O ./end2end_stealing/pretrained_weights/checkpoint_0099.pth.tar
